<h1>TMDB <h1>
    

    - Part 1: Exploring and Preparing the data for analysis
    - Part 2: Analyses of different genres
    - Part 3: 

<h2> Part 1: Exploring and  preparing the data for analysis:  </h2> 
 We start with the import of packages we will eventually need. Furthermore, we import the datasets and start with exploring and preparing the data for further analysis.

In [ ]:
import json

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.decomposition import PCA # Principal Component Analysis module
from sklearn.cluster import KMeans # KMeans clustering 


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

movies = pd.read_csv('../input/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdb_5000_credits.csv')




Let's just start with some easy questions to get familiar with the data. So what does the data look like? We'll start with taking a look at the movies data frame.

In [ ]:
movies.head()

The first thing we notice is that the columns are a bit in an awkward order to take a fine look at the data. A preferable first column of this data frame, would, for example, be the title of the movie and not the movie's budget. 

We also notice that the columns 'genres', 'keywords', 'production_companies', 'production_countries' and 'spoken_languages' are of the dictionary type, so right now they are quite hard to read, but later on we will find a way to work with them.

Amongst the numerical columns, there's a movie budget, a movie ID, popularity, revenue, runtime, a vote average and the amount of votes a movie has received. 

A good description of what the popularity variable should be telling us, is no where to be found, so it will be hard to use this column for our predictions later on. Besides the fact that the ID column is numerical, it is also not of interest for making predictions about, for example, the revenue of a movie. For now, we leave this data frame as it is and we'll take a quick look at the other one.

In [ ]:
credits.head()

So this data frame has way fewer columns. The cast and crew might be interesting later on. Since this data frame contains only two extra columns, we'll try to merge it with the  movies data frame. If they are in the same order, we can just concatenate the data frames, so let's see if in both data frames every row is about the same movie:

In [ ]:
(credits['title']==movies['title']).describe()

This tells us that every row in the credits data base has the same movie title as the same row in the movies data base. To prevent getting duplicate columns, we'll remove the movie_id and title column from the credits data frame and concatenate them.

In [ ]:
del credits['title']
del credits['movie_id']
movie_df = pd.concat([movies, credits], axis=1)

In [ ]:
movie_df.head()

The concatenation worked. However, the columns are a bit in an awkward order and columns like homepage aren't that interesting for us. We choose the interesting columns, put them in a nice order and create a new data frame

In [ ]:
newCols = ['id','title','release_date','popularity','vote_average','vote_count',
           'budget','revenue','genres','keywords','cast','crew','tagline', 'runtime', 'production_companies', 
           'production_countries', 'status']

df2 = movie_df[newCols]
df2.head()

Let's explore our data frame a bit more in depth, let's take a look at our numerical columns.

In [ ]:
df2.describe().round()

Note that runtime consists of a few empty values, before we can really work with our data frame, we need to solve this. We use an imputer for this:

In [ ]:
my_imputer = Imputer()

temp=df2
X2 = my_imputer.fit_transform(df2[['runtime']])
df2['runtime'] = X2
df2.describe().round()

So now at least all the numerical columns are complete. Let's take a quick look at how all the variables are distributed.

In [ ]:
del df2['id']

In [ ]:
#df2['vote_classes'] = pd.cut(df2['vote_average'],10, labels=["1", "2","3","4","5","6","7","8","9","10"])
df2['vote_classes'] = pd.cut(df2['vote_average'],4, labels=["low", "medium-low","medium-high","high"])

In [ ]:
fig = plt.figure(figsize = (10,15))
ax = fig.gca()

#fig, axes = plt.subplots(nrows=3, ncols=2)
#fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

#fig.subplots_adjust(hspace=0.1)
df2.hist(ax=ax)
#df2.hist(ax=ax)

Note that everything is quite skewed. We'll try getting more in depth into this later.

<h2> Part 3: Analyze genres: <h2>

Now that we've got a good overview of the distribution of our numerical variables, let's take a closer look at our non-numerical variables. We choose to start with looking at the genres, since this variable has got the least variability, should be the most easy target for analysis.

The genres column contains variables of the string type, while they are in dictionaries. Moreover, the colomn is a json column. To analyse and understand the data it is necessary to change the type of the variable and filter the columns.
Despite the fact that we already loaded our data for the exploration, we'll reload it here and make sure to load the json columns correctly. To do this, we made use of a few tricks found in another Kernel*

In [ ]:
def load_tmdb_movies(path):
    df = pd.read_csv(path)
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.date())
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

def load_tmdb_credits(path):
    df = pd.read_csv(path)
    json_columns = ['cast', 'crew']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])

credits = load_tmdb_credits("../input/tmdb_5000_credits.csv")
movies = load_tmdb_movies("../input/tmdb_5000_movies.csv")

del credits['title']
df = pd.concat([movies, credits], axis=1)

df['genres'] = df['genres'].apply(pipe_flatten_names)

liste_genres = set()
for s in df['genres'].str.split('|'):
    liste_genres = set().union(s, liste_genres)
liste_genres = list(liste_genres)
liste_genres.remove('')


So what happened here is the following: first, we changed the type of the genres variable. Aferwards, we made use of the structure of the column and the *split()* function.  Because the genre always appears after the word *name*, we were able to filter out al the words after the word name and create a list of every genre that occurs in the genre-column.

Now, let's reduce our data frame. To get more insight about the influence of a movie's genre, title, vote_average, release_data, runtime, budget and revenue are the most import important variables. We also add a column for every genre, containing only 1s and 0s whether a movie is of a specific genre or not.  

In [ ]:
df_reduced = df[['title','vote_average','release_date','runtime','budget','revenue']].reset_index(drop=True)

for genre in liste_genres:
    df_reduced[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)
df_reduced[:5]

df_reduced.head()

Now that we've got an easy to work with data frame for the movie genres, we can take a look to the distribution of the genres by creating a pie chart*. 

In [ ]:
plt.rc('font', weight='bold')
f, ax = plt.subplots(figsize=(5,5))
genre_count = []
for genre in liste_genres:
    genre_count.append([genre, df_reduced[genre].values.sum()])
genre_count.sort(key = lambda x:x[1], reverse = True)
labels, sizes = zip(*genre_count)
labels_selected = [n if v > sum(sizes) * 0.01 else '' for n, v in genre_count]
ax.pie(sizes, labels=labels_selected,
      autopct = lambda x:'{:2.0f}%'.format(x) if x>1 else '',
      shadow = False, startangle=0)
ax.axis('equal')
plt.tight_layout()

This pie chart shows which genres are most common in the movies dataset.We find that drama movies are most common, followed by comedy. Afterwards, thriller and action movies are the most popular. Interestingly, half of the movies is from the top 5 genres. (51%). This suggest that the main genre of the most movies are drama, comedy, thriller, action. However, the top 5 most common genres could be seen as more general descriptions. For example, movies with the genre war might also be tagged as action movies or drama movies.

Now let's try to get a more in depth view of the genres. In this cell we calculate the average votes, budget, and revenue for the different genres. we create a new data frame consisiting of every genre and the calculated averages. **

In [ ]:
mean_per_genre = pd.DataFrame(liste_genres)

#Mean votes average
newArray = []*len(liste_genres)
for genre in liste_genres:
    newArray.append(df_reduced.groupby(genre, as_index=True)['vote_average'].mean())
newArray2 = []*len(liste_genres)
for i in range(len(liste_genres)):
    newArray2.append(newArray[i][1])

mean_per_genre['mean_votes_average']=newArray2

#Mean budget
newArray = []*len(liste_genres)
for genre in liste_genres:
    newArray.append(df_reduced.groupby(genre, as_index=True)['budget'].mean())
newArray2 = []*len(liste_genres)
for i in range(len(liste_genres)):
    newArray2.append(newArray[i][1])

mean_per_genre['mean_budget']=newArray2

#Mean revenue 
newArray = []*len(liste_genres)
for genre in liste_genres:
    newArray.append(df_reduced.groupby(genre, as_index=True)['revenue'].mean())
newArray2 = []*len(liste_genres)
for i in range(len(liste_genres)):
    newArray2.append(newArray[i][1])

mean_per_genre['mean_revenue']=newArray2

mean_per_genre['profit'] = mean_per_genre['mean_revenue']-mean_per_genre['mean_budget']

mean_per_genre    

Let's see which genres are the best scoring ones in each category:

In [ ]:
mean_per_genre.sort_values('mean_votes_average', ascending=False).head()


In [ ]:
mean_per_genre.sort_values('mean_budget', ascending=False).head()

In [ ]:
mean_per_genre.sort_values('mean_revenue', ascending=False).head()

In [ ]:
mean_per_genre.sort_values('profit', ascending=False).head()

It's very interesting to see that the top 5 highest vote average consists of *History, War, Drama, Music* and *Foreign*, while none of these genres are in either one of the other three categories, which all have the same top 3: *Animation, Adventure, Fantasy*. On the one hand, this is easily explained, since budget and revenue should be closely elated and profit is directly derived from budget and revenue. However, we would have expected a higher correlation between the budget and the quality of a movie.

To go even more in depth, we want to analyse the averages per genre per year.  Therefore, we first extend the dataframe. with the year of release per movie.  Afterwards, we create a new dataframe which contains the average votes, average runtime, and average budget per release year and per genre. 

In the last step in the cell below, only the rows that contain a 1 for genre are kept, so we create a data frame with only the specific genres. 

In [ ]:
from datetime import datetime

t = df_reduced['release_date']
t = pd.to_datetime(t)
t = t.dt.year
df_reduced['release_year'] = t

df_list = []*len(liste_genres)
for genre in liste_genres:
    df_list.append(df_reduced.groupby([genre,'release_year']).mean().reset_index())

df_per_genre = []*len(liste_genres)
for i in range(len(df_list)):
    df_per_genre.append(df_list[i][df_list[i].ix[:,0] == 1])


Now we create tables which contain the average budget, average revenue, and average votes per year per genre. We start with creating a new table with the cloumns 1988 till 2017. Afterwards, the data for the different variables is implemented. **

In [ ]:
# Budget
columns = range(1988,2018)
budget_genre = pd.DataFrame( columns = columns)

for genre in liste_genres:
    temp=(df_per_genre[liste_genres.index(genre)].pivot_table(index = genre, columns = 'release_year', values = 'budget', aggfunc = np.mean))
    temp = temp[temp.columns[-30:]].loc[1]
    budget_genre.loc[liste_genres.index(genre)]=temp
budget_genre['genre']=liste_genres

# Revenue 

columns = range(1988,2018)
revenue_genre = pd.DataFrame( columns = columns)

for genre in liste_genres:
    temp=(df_per_genre[liste_genres.index(genre)].pivot_table(index = genre, columns = 'release_year', values = 'revenue', aggfunc = np.mean))
    temp = temp[temp.columns[-30:]].loc[1]
    revenue_genre.loc[liste_genres.index(genre)]=temp
revenue_genre['genre']=liste_genres

# Vote average 
columns = range(1988,2018)
vote_avg_genre = pd.DataFrame( columns = columns)

for genre in liste_genres:
    temp=(df_per_genre[liste_genres.index(genre)].pivot_table(index = genre, columns = 'release_year', values = 'vote_average', aggfunc = np.mean))
    temp = temp[temp.columns[-30:]].loc[1]
    vote_avg_genre.loc[liste_genres.index(genre)]=temp
vote_avg_genre['genre']=liste_genres

#vote_avg_genre.index = vote_avg_genre['genre']

Let's take a look at the data frames we generated.

### Mean budget per genre per year:

In [ ]:
budget_genre.index = budget_genre['genre']
budget_genre

### Mean revenue per genre per year:

In [ ]:
revenue_genre.index = revenue_genre['genre']
revenue_genre


### Mean vote average per genre per year:

In [ ]:
vote_avg_genre.index = vote_avg_genre['genre']
vote_avg_genre

We can create more insight in these tables by making heatmaps**. 

### Budget:

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
cmap = sns.cubehelix_palette(start = 1.5, rot = 1.5, as_cmap = True)
sns.heatmap(budget_genre.ix[:,0:30], xticklabels=3, cmap=cmap, linewidths=0.05)

The heatmap shows that in general, movies had  an increasing budget over the years. Especially the genres Fantasy, advernture, family, action, science fiction, and animation. The heatmap also shows that Western movies had an extremely high budget in 2013. This could mean that a costly movie is produced in 2013 which has great influence on the average.  We might later on remove this possible outlier, to get a better overview of the distribution of the rest of the movies.

## Revenue:

In [ ]:

fig, ax = plt.subplots(figsize=(9,9))
cmap = sns.cubehelix_palette(start = 1.5, rot = 1.5, as_cmap = True)
sns.heatmap(revenue_genre.ix[:,0:30], xticklabels=3, cmap=cmap, linewidths=0.05)

This heatmap shows the average revenue of genres from 1988 till 2017. The most clear increase of average is in the genres fantasy, adventure, family, action, science fiction. Interestingly, the graph shows that the revenues of the genre animation are colored black in 1994. This is surprisingly because there are no black colored revenues in the graph and in general revenues are lower in 1994 than movies that are produced in later years.  A reason for this could be that there are only a few movies in the genre animation in 1994 and that those movies did extremely well.  The previous heatmap does not show an above average budget for animation movies in 1994. 


## Vote average:

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
cmap = sns.cubehelix_palette(start = 1.5, rot = 1.5, as_cmap = True)
sns.heatmap(vote_avg_genre.ix[:,0:30], xticklabels=3, cmap=cmap, linewidths=0.05)

This heatmap is way darker than the previous two, which suggests that the average is relatively higher than in the other two categories. Most of the categories seem to be getting somewhere around a 6 out of 10 score. Especially notable is the fact that there are very few green or orange colored cells, which should mean that the most movies are on average just a nice watch.

As said before, we would like to remove the very high budget input from the Western genre, to make the heatmap less skewed. Let's see what happens:

In [ ]:
temp = budget_genre
temp[2013]=temp[2013].replace(2.550000e+08, 0)

This heatmap obviously shows that Fantasy Adventure, Science Fiction, and Animation have on average the highest budget. It is also clear that movies had an increasing budget over the years. However, there are a few exceptions. For example  Western movies had an above average budget in 2004 and history in 2000. This might be an effect of individual movies with a high budget. 

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
cmap = sns.cubehelix_palette(start = 1.5, rot = 1.5, as_cmap = True)
sns.heatmap(temp.ix[:,0:30], xticklabels=3, cmap=cmap, linewidths=0.05)

## It might also be nice to create a visualisation on how many times different genres are connected with each other. So which genres occur the most together in the same movie, but this is something for later on.

# Numerical Analysis

So let's take a closer look at our the numerical columns in our data frame.  Let's start by creating a data frame containing only numbered columns.

In [ ]:
num_list = ['budget','popularity','revenue','runtime','vote_average','vote_count']
movie_num = df2[num_list]
movie_num.head()

Let's take a look at how everything is correlated:

In [ ]:
f, ax = plt.subplots(figsize=(12,10))
plt.title('Pearson Correlation of Movie Features')
sns.heatmap(movie_num.astype(float).corr(), linewidths=0.25, vmax=1.0, square=True,
           cmap="YlGnBu", linecolor='black', annot=True)

We see quite a few dark/blue squares. These are the higher correlated variables. To be able to make predictions about certain movies later on, this might be some important knowledge.

# Comparing different regression techniques

We want to compare a few regression techniques to help us in making predictions. We'll use linear regression and random forest, as treated in the lectures.
We start by recreating our numerical data frame.

In [ ]:
num_list = ['budget','popularity','revenue','runtime','vote_average','vote_count']
movie_num = df2[num_list]
movie_num.head()

We want the vote_average to be our target values, budget, popularity, revenue, runtime and vote_count are trainng values.

In [ ]:
training_list = ['budget','popularity','revenue','runtime','vote_count']
training = movie_num[training_list]
target = movie_num['vote_average']

In [ ]:
X = training.values
y = target.values

We split our data in a train and a test frame.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

Now let's train a linear regression model and plot the results: \***

In [ ]:
from sklearn import linear_model
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred_lr = regr.predict(X_test)

In [ ]:
f = plt.figure(figsize=(10,5))
plt.scatter(X_test[:,1], y_test, s=50,label="Real vote_average");
plt.scatter(X_test[:,1], y_pred_lr,s=100, c='r',label="Predicted vote_average");
plt.ylabel("vote_average");
plt.legend(loc=2);

Now let's see what happens if we use a random forest regression model:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Create linear regression object
rf = RandomForestRegressor(1)

# Train the model using the training sets
rf.fit(X_train, y_train)

# Make predictions using the testing set
y_pred_rf = rf.predict(X_test)

In [ ]:
f = plt.figure(figsize=(10,5))
plt.scatter(X_test[:,1], y_test, s=50,label="Real vote_average");
plt.scatter(X_test[:,1], y_pred_rf,s=100, c='r',label="Predited vote_average");
plt.ylabel("vote_average");
plt.legend(loc=2);

And let's compare them:

In [ ]:
from sklearn.metrics import mean_squared_error

error_lr = mean_squared_error(y_test,y_pred_lr)
error_rf = mean_squared_error(y_test,y_pred_rf)

print(error_lr)
print(error_rf)

In [ ]:
f = plt.figure(figsize=(10,5))
plt.bar(range(2),[error_lr,error_rf])
plt.xlabel("Classifiers");
plt.ylabel("Mean Squared Error of the vote_average");
plt.xticks(range(2),['Linear Regression','Random Forest'])
plt.legend(loc=2);

So the mean squared error for the random forest regression is a little higher than for the linear regression, but both estimators seem to be very decent.

\* https://www.kaggle.com/fabiendaniel/categorizing-actors-hands-on-plotly <br>
\** https://www.kaggle.com/diegoinacio/imdb-genre-based-analysis <br>
\*** introduction to data science, week 4, Comparison of Regression Techniques on House prediction prices.ipynb

**ROBBERT**

In [ ]:

import nltk
from nltk.corpus import wordnet
PS = nltk.stem.PorterStemmer()

import plotly.offline as pyo
pyo.init_notebook_mode()
from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

# Country Analysis<br>
We are interested in the following question: What countries produce the most movies?[](http://)


In [ ]:
def load_tmdb_movies(path):
    df = pd.read_csv(path)
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.date())
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df
#____________________________
def load_tmdb_credits(path):
    df = pd.read_csv(path)
    json_columns = ['cast', 'crew']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df
#_______________________________________
def safe_access(container, index_values):
    result = container
    try:
        for idx in index_values:
            result = result[idx]
        return result
    except IndexError or KeyError:
        return pd.np.nan
#_______________________________________
LOST_COLUMNS = [
    'actor_1_facebook_likes',
    'actor_2_facebook_likes',
    'actor_3_facebook_likes',
    'aspect_ratio',
    'cast_total_facebook_likes',
    'color',
    'content_rating',
    'director_facebook_likes',
    'facenumber_in_poster',
    'movie_facebook_likes',
    'movie_imdb_link',
    'num_critic_for_reviews',
    'num_user_for_reviews']
#_______________________________________
TMDB_TO_IMDB_SIMPLE_EQUIVALENCIES = {
    'budget': 'budget',
    'genres': 'genres',
    'revenue': 'revenue',
    'title': 'movie_title',
    'runtime': 'duration',
    'original_language': 'language',  
    'keywords': 'plot_keywords',
    'vote_count': 'num_voted_users'}
#_______________________________________     
IMDB_COLUMNS_TO_REMAP = {'imdb_score': 'vote_average'}
#_______________________________________
def get_director(crew_data):
    directors = [x['name'] for x in crew_data if x['job'] == 'Director']
    return safe_access(directors, [0])
#_______________________________________
def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])
#_______________________________________
def convert_to_original_format(movies, credits):
    tmdb_movies = movies.copy()
    tmdb_movies.rename(columns=TMDB_TO_IMDB_SIMPLE_EQUIVALENCIES, inplace=True)
    tmdb_movies['title_year'] = pd.to_datetime(tmdb_movies['release_date']).apply(lambda x: x.year)
    # I'm assuming that the first production country is equivalent, but have not been able to validate this
    tmdb_movies['country'] = tmdb_movies['production_countries'].apply(lambda x: safe_access(x, [0, 'name']))
    tmdb_movies['language'] = tmdb_movies['spoken_languages'].apply(lambda x: safe_access(x, [0, 'name']))
    tmdb_movies['director_name'] = credits['crew'].apply(get_director)
    tmdb_movies['actor_1_name'] = credits['cast'].apply(lambda x: safe_access(x, [1, 'name']))
    tmdb_movies['actor_2_name'] = credits['cast'].apply(lambda x: safe_access(x, [2, 'name']))
    tmdb_movies['actor_3_name'] = credits['cast'].apply(lambda x: safe_access(x, [3, 'name']))
    tmdb_movies['genres'] = tmdb_movies['genres'].apply(pipe_flatten_names)
    tmdb_movies['plot_keywords'] = tmdb_movies['plot_keywords'].apply(pipe_flatten_names)
    return tmdb_movies

In [ ]:
#______________
# the packages
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
#___________________
# and the dataframe
credits = load_tmdb_credits("../input/tmdb_5000_credits.csv")
movies = load_tmdb_movies("../input/tmdb_5000_movies.csv")
df = convert_to_original_format(movies, credits)
#___________________________
# countries in the dataframe
df['country'].unique()


In [ ]:
df['country'].describe()

In [ ]:
#Extract number of films per country
df_countries = df['title_year'].groupby(df['country']).count()
df_countries = df_countries.reset_index()
df_countries.rename(columns ={'title_year':'count'}, inplace = True)
df_countries = df_countries.sort_values('count', ascending = False)
df_countries.reset_index(drop=True, inplace = True)

#Create pie chart with number of films per country
sns.set_context("poster", font_scale=0.6)
plt.rc('font', weight='bold')
f, ax = plt.subplots(figsize=(11, 6))
labels = [s[0] if s[1] > 80 else ' ' 
          for index, s in  df_countries[['country', 'count']].iterrows()]
sizes  = df_countries['count'].values
explode = [0.0 if sizes[i] < 100 else 0.0 for i in range(len(df_countries))]
ax.pie(sizes, explode = explode, labels = labels,
       autopct = lambda x:'{:1.0f}%'.format(x) if x > 1 else '',
       shadow=False, startangle=45)
ax.axis('equal')
ax.set_title('% of films per country',
             bbox={'facecolor':'k', 'pad':5},color='w', fontsize=16);

As one would have expected, most of the movies were created in the united states.

In [ ]:
#Create a chloropleth map
data = dict(type='choropleth',
locations = df_countries['country'],
locationmode = 'country names', z = df_countries['count'],
text = df_countries['country'], colorbar = {'title':'Films nb.'},
colorscale=[[0, 'rgb(224,255,255)'],
            [0.01, 'rgb(166,206,227)'], [0.02, 'rgb(31,120,180)'],
            [0.03, 'rgb(178,223,138)'], [0.05, 'rgb(51,160,44)'],
            [0.10, 'rgb(251,154,153)'], [0.20, 'rgb(255,255,0)'],
            [1, 'rgb(227,26,28)']],    
reversescale = False)

layout = dict(title='Number of films in the TMDB database',
geo = dict(showframe = True, projection={'type':'Mercator'}))

choromap = go.Figure(data = [data], layout = layout)
iplot(choromap, validate=False)

# Keyword Analysis
### Part 1: Cleaning the data
For the analysis of the keywords, we will be using the same method we applied in the genre analysis. However, before it is possible for us to analyze the data, it needs a lot of cleaning. To clean the data, we use the same method as applied in this notebook: https://www.kaggle.com/fabiendaniel/film-recommendation-engine.

In [ ]:
def load_tmdb_movies(path):
    df = pd.read_csv(path)
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.date())
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

def load_tmdb_credits(path):
    df = pd.read_csv(path)
    json_columns = ['cast', 'crew']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])

credits = load_tmdb_credits("../input/tmdb_5000_credits.csv")
movies = load_tmdb_movies("../input/tmdb_5000_movies.csv")

del credits['title']
df = pd.concat([movies, credits], axis=1)

df['keywords'] = df['keywords'].apply(pipe_flatten_names)

liste_keywords = set()
for s in df['keywords'].str.split('|'):
    liste_keywords = set().union(s, liste_keywords)
liste_keywords = list(liste_keywords)
liste_keywords.remove('')

We are interested in which keywords occur the most in our dataset. We use the following function to count them.


In [ ]:
def count_word(df, ref_col, liste):
    keyword_count = dict()
    for s in liste: keyword_count[s] = 0
    for liste_keywords in df[ref_col].str.split('|'):        
        if type(liste_keywords) == float and pd.isnull(liste_keywords): continue        
        for s in [s for s in liste_keywords if s in liste]: 
            if pd.notnull(s): keyword_count[s] += 1
    #______________________________________________________________________
    # convert the dictionary in a list to sort the keywords by frequency
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count

keyword_occurences, dum = count_word(df, 'keywords', liste_keywords)
keyword_occurences[:5]

In [ ]:
# Collect the keywords
def keywords_inventory(dataframe, colonne = 'keywords'):
    PS = nltk.stem.PorterStemmer()
    keywords_roots  = dict()  # collect the words / root
    keywords_select = dict()  # association: root <-> keyword
    category_keys = []
    icount = 0
    for s in dataframe[colonne]:
        if pd.isnull(s): continue
        for t in s.split('|'):
            t = t.lower() ; racine = PS.stem(t)
            if racine in keywords_roots:                
                keywords_roots[racine].add(t)
            else:
                keywords_roots[racine] = {t}
    
    for s in keywords_roots.keys():
        if len(keywords_roots[s]) > 1:  
            min_length = 1000
            for k in keywords_roots[s]:
                if len(k) < min_length:
                    clef = k ; min_length = len(k)            
            category_keys.append(clef)
            keywords_select[s] = clef
        else:
            category_keys.append(list(keywords_roots[s])[0])
            keywords_select[s] = list(keywords_roots[s])[0]
                   
    print("Nb of keywords in variable '{}': {}".format(colonne,len(category_keys)))
    return category_keys, keywords_roots, keywords_select

keywords, keywords_roots, keywords_select = keywords_inventory(df, colonne = 'keywords')

Of course, different movies use different keywords for their movies. A problem is, that often a lot of those keywords are the same, although they are communicated in a different form by the different movie producers. The function above inventorizes the different keywords using nltk. The package identifies the 'roots' of different words and groups the different words according to its root. Then, we can replace the words that have a common root with their root. In this way, similar words that are phrased differently are assigned a common 'root'.

When executing the function, it also shows the amount of different keywords, 9474 in our case.

### Roots

In [ ]:
# Plot of a sample of keywords that appear in close varieties
# Different words with similar roots
icount = 0
for s in keywords_roots.keys():
    if len(keywords_roots[s]) > 1: 
        icount += 1
        if icount < 15: print(icount, keywords_roots[s], len(keywords_roots[s]))

In [ ]:
#Each different form of a word will be replaced by their roots

def remplacement_df_keywords(df, dico_remplacement, roots = False):
    df_new = df.copy(deep = True)
    for index, row in df_new.iterrows():
        chaine = row['keywords']
        if pd.isnull(chaine): continue
        nouvelle_liste = []
        for s in chaine.split('|'): 
            clef = PS.stem(s) if roots else s
            if clef in dico_remplacement.keys():
                nouvelle_liste.append(dico_remplacement[clef])
            else:
                nouvelle_liste.append(s)       
        df_new.set_value(index, 'keywords', '|'.join(nouvelle_liste)) 
    return df_new

#The cleaned keywords will be stored in a new dataframe
df_keywords_cleaned = remplacement_df_keywords(df, keywords_select, roots = True)

### Synonyms
We use the nltk package to get rid of synonyms. 

In [ ]:
#The function below takes a word as a parameter and returns all of the synonyms of that word according to the nltk package.
def get_synonymes(word):
    lemma = set()
    for ss in wordnet.synsets(word):
        for w in ss.lemma_names():
            # We just get the 'nouns':
            index = ss.name().find('.')+1
            if ss.name()[index] == 'n': lemma.add(w.lower().replace('_',' '))
    return lemma  
#__________________________________________________________________________
def test_keyword(mot, key_count, threshold):
    return (False , True)[key_count.get(mot, 0) >= threshold]

#__________________________________________________________________________
keyword_occurences.sort(key = lambda x:x[1], reverse = False)
key_count = dict()
for s in keyword_occurences:
    key_count[s[0]] = s[1]

# Creation of a dictionary to replace keywords by higher frequency keywords
remplacement_mot = dict()
icount = 0
for index, [mot, nb_apparitions] in enumerate(keyword_occurences):
    if nb_apparitions > 5: continue  # only the keywords that appear less than 5 times
    lemma = get_synonymes(mot)
    if len(lemma) == 0: continue     # case of the plurals
    liste_mots = [(s, key_count[s]) for s in lemma 
                  if test_keyword(s, key_count, key_count[mot])]
    liste_mots.sort(key = lambda x:(x[1],x[0]), reverse = True)    
    if len(liste_mots) <= 1: continue       # no replacement
    if mot == liste_mots[0][0]: continue    # replacement by himself
    icount += 1
    if  icount < 8:
        print('{:<12} -> {:<12} (init: {})'.format(mot, liste_mots[0][0], liste_mots))    
    remplacement_mot[mot] = liste_mots[0][0]

print(90*'_'+'\n'+'The replacement concerns {}% of the keywords.'
      .format(round(len(remplacement_mot)/len(keywords)*100,2)))


In [ ]:
# 2 successive replacements
print('Keywords that appear both in keys and values:'.upper()+'\n'+45*'-')
icount = 0
for s in remplacement_mot.values():
    if s in remplacement_mot.keys():
        icount += 1
        if icount < 10: print('{:<20} -> {:<20}'.format(s, remplacement_mot[s]))

for key, value in remplacement_mot.items():
    if value in remplacement_mot.keys():
        remplacement_mot[key] = remplacement_mot[value]

In [ ]:
# replacement of keyword varieties by the main keyword
df_keywords_synonyms = remplacement_df_keywords(df_keywords_cleaned, remplacement_mot, roots = False)   
keywords, keywords_roots, keywords_select = keywords_inventory(df_keywords_synonyms, colonne = 'keywords')

In [ ]:
# New count of keyword occurences
keywords.remove('')
new_keyword_occurences, keywords_count = count_word(df_keywords_synonyms,'keywords',keywords)
new_keyword_occurences[:5]

In [ ]:
# deletion of keywords with low frequencies
def remplacement_df_low_frequency_keywords(df, keyword_occurences):
    df_new = df.copy(deep = True)
    key_count = dict()
    for s in keyword_occurences: 
        key_count[s[0]] = s[1]    
    for index, row in df_new.iterrows():
        chaine = row['keywords']
        if pd.isnull(chaine): continue
        nouvelle_liste = []
        for s in chaine.split('|'): 
            if key_count.get(s, 4) > 3: nouvelle_liste.append(s)
        df_new.set_value(index, 'keywords', '|'.join(nouvelle_liste))
    return df_new

# Creation of a dataframe where keywords of low frequencies are suppressed
df_keywords_occurence = remplacement_df_low_frequency_keywords(df_keywords_synonyms, new_keyword_occurences)
df2 = df_keywords_occurence
keywords, keywords_roots, keywords_select = keywords_inventory(df2, colonne = 'keywords')   

We see a drastic decrease in different keywords

In [ ]:
#Show the 50 most occuring keywords in a histogram
fig = plt.figure(1, figsize=(18,13))
trunc_occurences = new_keyword_occurences[0:50]
# LOWER PANEL: HISTOGRAMS
ax2 = fig.add_subplot(2,1,2)
y_axis = [i[1] for i in trunc_occurences]
x_axis = [k for k,i in enumerate(trunc_occurences)]
x_label = [i[0] for i in trunc_occurences]
plt.xticks(rotation=85, fontsize = 15)
plt.yticks(fontsize = 15)
plt.xticks(x_axis, x_label)
plt.ylabel("Nb. of occurences", fontsize = 18, labelpad = 10)
ax2.bar(x_axis, y_axis, align = 'center', color='g')
#_______________________
plt.title("Keywords popularity",bbox={'facecolor':'k', 'pad':5},color='w',fontsize = 25)
plt.show()

### Part 2: Analysis

For our analysis of the keywords, not every single column is relevent, so we remove a couple of columns from the dataset, the same way we did before. We also add a column for every single keyword, containing a 1 or 0 depending on whether that keyword belongs to a specific movie or not.

Then, we are interested in computing the means of several categories for all the keywords, the same way as we did with the genres.


In [ ]:
liste_keywords = set()
for s in df2['keywords'].str.split('|'):
    liste_keywords = set().union(s, liste_keywords)
liste_keywords = list(liste_keywords)
liste_keywords.remove('')

df_reduced = df2[['title','vote_average','release_date','runtime','budget','revenue']].reset_index(drop=True)

for keywords in liste_keywords:
    df_reduced[keywords] = df2['keywords'].str.contains(keywords).apply(lambda x:1 if x else 0)
df_reduced.head()


In [ ]:
#Brackets in the name of the keyword are not allowed in our way of computing the mean --> remove them.
#We identified three keywords containing brackets, we remove them as follows:
liste_keywords.remove('national security agency (nsa)')
liste_keywords.remove('middle-earth (tolkien)')
liste_keywords.remove('lover (female)')

Now, we use the exact same way as we did with the genres to compute several averages for the keywords.

In [ ]:
mean_per_keywords = pd.DataFrame(liste_keywords)

#Mean votes average
newArray = []*len(liste_keywords)
for keywords in liste_keywords:
    newArray.append(df_reduced.groupby(keywords, as_index=True)['vote_average'].mean())
newArray2 = []*len(liste_keywords)
for i in range(len(liste_keywords)):
    # print(newArray[i][1], i)
    newArray2.append(newArray[i][1])

mean_per_keywords['mean_votes_average']=newArray2

#Mean budget
newArray = []*len(liste_keywords)
for keywords in liste_keywords:
    newArray.append(df_reduced.groupby(keywords, as_index=True)['budget'].mean())
newArray2 = []*len(liste_keywords)
for i in range(len(liste_keywords)):
    newArray2.append(newArray[i][1])

mean_per_keywords['mean_budget']=newArray2

#Mean revenue 
newArray = []*len(liste_keywords)
for keywords in liste_keywords:
    newArray.append(df_reduced.groupby(keywords, as_index=True)['revenue'].mean())
newArray2 = []*len(liste_keywords)
for i in range(len(liste_keywords)):
    newArray2.append(newArray[i][1])

mean_per_keywords['mean_revenue']=newArray2

mean_per_keywords['profit'] = mean_per_keywords['mean_revenue']-mean_per_keywords['mean_budget']

mean_per_keywords.head()

For every category the five largerst mean are shown.

In [ ]:
mean_per_keywords.sort_values('mean_votes_average', ascending=False).head()

In [ ]:
mean_per_keywords.sort_values('mean_budget', ascending=False).head()

In [ ]:
mean_per_keywords.sort_values('mean_revenue', ascending=False).head()

In [ ]:
mean_per_keywords.sort_values('profit', ascending=False).head()

We can further visualize our findings using bar plots. 


In [ ]:
#In order to to get a good visualisation we restructure our dataframe a little bit.
mean_per_keywords.index = mean_per_keywords.loc[:,0]
mean_per_keywords = mean_per_keywords.drop(0,axis=1)

In [ ]:
df = mean_per_keywords.sort_values('mean_votes_average', ascending=False)
df = df[0:50]
fig = plt.figure(1, figsize=(18,13))

import matplotlib.pyplot as plt
ax = df['mean_votes_average'].plot(kind='bar', title ="mean_vote_average", figsize=(15, 4), legend=True, fontsize=12, color='green')
ax.set_xlabel("Keyword", fontsize=12)
ax.set_ylabel("mean_votes_average", fontsize=12 )
ax.set_ylim([7.2,7.7])
plt.show()

In [ ]:
df = mean_per_keywords.sort_values('mean_budget', ascending=False)
df = df[0:50]
fig = plt.figure(1, figsize=(18,13))

import matplotlib.pyplot as plt
ax = df['mean_budget'].plot(kind='bar', title ="mean_budget", figsize=(15, 4), legend=True, fontsize=12, color='red')
ax.set_xlabel("Keyword", fontsize=12)
ax.set_ylabel("mean_budget", fontsize=12)
ax.set_ylim([1e+08 , 2.2e+08])
plt.show()

In [ ]:
df = mean_per_keywords.sort_values('mean_revenue', ascending=False)
df = df[0:50]
fig = plt.figure(1, figsize=(18,13))

import matplotlib.pyplot as plt
ax = df['mean_revenue'].plot(kind='bar', title ="mean_revenue", figsize=(15, 4), legend=True, fontsize=12, color='blue')
ax.set_xlabel("Keyword", fontsize=12)
ax.set_ylabel("mean_revenue", fontsize=12)
ax.set_ylim([4e+08 , 10e+08])
plt.show()

In [ ]:
df = mean_per_keywords.sort_values('profit', ascending=False)
df = df[0:50]
fig = plt.figure(1, figsize=(18,13))

import matplotlib.pyplot as plt
ax = df['profit'].plot(kind='bar', title ="Profit", figsize=(15, 4), legend=True, fontsize=12, color='pink')
ax.set_xlabel("Keyword", fontsize=12)
ax.set_ylabel("Profit", fontsize=12)
ax.set_ylim([3e+08 , 8e+08])
plt.show()

## Cast Analysis
A previous version of this dataset only contained the top three actors per movie. Since we only want to analyze the most important actors of a movie and since the old dataset was a bit more suited to do that, we convert the dataset back to its previous state using Sohier Dane's method.

In [ ]:
# Columns that existed in the IMDB version of the dataset and are gone.
LOST_COLUMNS = [
    'actor_1_facebook_likes',
    'actor_2_facebook_likes',
    'actor_3_facebook_likes',
    'aspect_ratio',
    'cast_total_facebook_likes',
    'color',
    'content_rating',
    'director_facebook_likes',
    'facenumber_in_poster',
    'movie_facebook_likes',
    'movie_imdb_link',
    'num_critic_for_reviews',
    'num_user_for_reviews'
                ]

# Columns in TMDb that had direct equivalents in the IMDB version. 
# These columns can be used with old kernels just by changing the names
TMDB_TO_IMDB_SIMPLE_EQUIVALENCIES = {
    'budget': 'budget',
    'genres': 'genres',
    'revenue': 'revenue',
    'title': 'movie_title',
    'runtime': 'duration',
    'original_language': 'language',  # it's possible that spoken_languages would be a better match
    'keywords': 'plot_keywords',
    'vote_count': 'num_voted_users',
                                         }

IMDB_COLUMNS_TO_REMAP = {'imdb_score': 'vote_average'}


def safe_access(container, index_values):
    # return a missing value rather than an error upon indexing/key failure
    result = container
    try:
        for idx in index_values:
            result = result[idx]
        return result
    except IndexError or KeyError:
        return pd.np.nan


def get_director(crew_data):
    directors = [x['name'] for x in crew_data if x['job'] == 'Director']
    return safe_access(directors, [0])


def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])


def convert_to_original_format(movies, credits):
    # Converts TMDb data to make it as compatible as possible with kernels built on the original version of the data.
    tmdb_movies = movies.copy()
    tmdb_movies.rename(columns=TMDB_TO_IMDB_SIMPLE_EQUIVALENCIES, inplace=True)
    tmdb_movies['title_year'] = pd.to_datetime(tmdb_movies['release_date']).apply(lambda x: x.year)
    # I'm assuming that the first production country is equivalent, but have not been able to validate this
    tmdb_movies['country'] = tmdb_movies['production_countries'].apply(lambda x: safe_access(x, [0, 'name']))
    tmdb_movies['language'] = tmdb_movies['spoken_languages'].apply(lambda x: safe_access(x, [0, 'name']))
    tmdb_movies['director_name'] = credits['crew'].apply(get_director)
    tmdb_movies['actor_1_name'] = credits['cast'].apply(lambda x: safe_access(x, [1, 'name']))
    tmdb_movies['actor_2_name'] = credits['cast'].apply(lambda x: safe_access(x, [2, 'name']))
    tmdb_movies['actor_3_name'] = credits['cast'].apply(lambda x: safe_access(x, [3, 'name']))
    tmdb_movies['genres'] = tmdb_movies['genres'].apply(pipe_flatten_names)
    tmdb_movies['plot_keywords'] = tmdb_movies['plot_keywords'].apply(pipe_flatten_names)
    return tmdb_movies

credits = load_tmdb_credits("../input/tmdb_5000_credits.csv")
movies = load_tmdb_movies("../input/tmdb_5000_movies.csv")
df = convert_to_original_format(movies, credits)

# We store a copy of the dataframe for later use
df3 = df

#Delete all the columns we won't be needing for this analysis.
columns = ['homepage', 'plot_keywords', 'language', 'overview', 'popularity', 'tagline',
           'original_title', 'num_voted_users', 'country', 'spoken_languages', 'duration',
          'production_companies', 'production_countries', 'status']

df = df.drop(columns, axis=1)



We are interested in the same descriptives for the actors, as we were for keywords and the genres. To do that, we first have to, once again, restructure the dataframe.

We first create a seperate dataframe for each of the three actors, after which we can combine them to get one dataframe with all three types of actor.

In [ ]:
liste_genres = set()
for s in df['genres'].str.split('|'):
    liste_genres = set().union(s, liste_genres)
liste_genres = list(liste_genres)
liste_genres.remove('')

df_reduced = df[['actor_1_name', 'vote_average',
                 'title_year', 'movie_title', 'revenue', 'budget']].reset_index(drop = True)
for genre in liste_genres:
    df_reduced[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)

df_reduced2 = df[['actor_2_name', 'vote_average',
                 'title_year', 'movie_title', 'revenue', 'budget']].reset_index(drop = True)
for genre in liste_genres:
    df_reduced2[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)

df_reduced3 = df[['actor_3_name', 'vote_average',
                 'title_year', 'movie_title', 'revenue', 'budget']].reset_index(drop = True)
for genre in liste_genres:
    df_reduced3[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)
    
#combine the three dataframes
df_reduced = df_reduced.rename(columns={'actor_1_name': 'actor'})
df_reduced2 = df_reduced2.rename(columns={'actor_2_name': 'actor'})
df_reduced3 = df_reduced3.rename(columns={'actor_3_name': 'actor'})

total = [df_reduced, df_reduced2, df_reduced3]
df_total = pd.concat(total)
df_total.head()

We compute averages for all actors in two categories: vote_average and title_year. We also compute an actors favorite genre.

In [ ]:
df_actors = df_total.groupby('actor').mean()
df_actors.loc[:, 'favored_genre'] = df_actors[liste_genres].idxmax(axis = 1)
df_actors.drop(liste_genres, axis = 1, inplace = True)
df_actors = df_actors.reset_index()

We expect the dataframe to contain a lot of actors that have only a single observation. These observation are likely to cause outliers if these observations are very extreme. We delete all actors that are linked to less than 5 movies in our dataframe.

In [ ]:
#The minimum amount of movies an actor plays in can easily be adapted by changing the selection
df_appearance = df_total[['actor', 'title_year']].groupby('actor').count()
df_appearance = df_appearance.reset_index(drop = True)
selection = df_appearance['title_year'] > 4
selection = selection.reset_index(drop = True)
most_prolific = df_actors[selection]

Now that we have a clear dataframe, let us show some descriptive statistics. We first sort the dataframe on all the different attributes from highest to lowest.

In [ ]:
most_prolific.sort_values('vote_average', ascending=False).head()

In [ ]:
most_prolific.sort_values('revenue', ascending=False).head()

In [ ]:
most_prolific.sort_values('budget', ascending=False).head()

We can now develop several plots to analyze our actors. Let us start by plotting the average budget per actor and the average revenue per actor.

In [ ]:
genre_count = []
for genre in liste_genres:
    genre_count.append([genre, df_reduced[genre].values.sum()])
genre_count.sort(key = lambda x:x[1], reverse = True)
labels, sizes = zip(*genre_count)
labels_selected = [n if v > sum(sizes) * 0.01 else '' for n, v in genre_count]
reduced_genre_list = labels[:19]
trace=[]
for genre in reduced_genre_list:
    trace.append({'type':'scatter',
                  'mode':'markers',
                  'y':most_prolific.loc[most_prolific['favored_genre']==genre,'revenue'],
                  'x':most_prolific.loc[most_prolific['favored_genre']==genre,'budget'],
                  'name':genre,
                  'text': most_prolific.loc[most_prolific['favored_genre']==genre,'actor'],
                  'marker':{'size':10,'opacity':0.7,
                            'line':{'width':1.25,'color':'black'}}})
layout={'title':'Actors favored genres',
       'xaxis':{'title':'mean year of activity'},
       'yaxis':{'title':'mean score'}}
fig=Figure(data=trace,layout=layout)
pyo.iplot(fig)

Except for some outliers, this looks like a very nice linear regression. This gives us the insight that overall, actors are worth their money. For most actors it is so, that if they play in a high budget movie, the movie is likely to have high revenue as well.

Next, let us look at the average vote an actor receives and their mean year of activity.

In [ ]:
reduced_genre_list = labels[:19]
trace=[]
for genre in reduced_genre_list:
    trace.append({'type':'scatter',
                  'mode':'markers',
                  'y':most_prolific.loc[most_prolific['favored_genre']==genre,'vote_average'],
                  'x':most_prolific.loc[most_prolific['favored_genre']==genre,'title_year'],
                  'name':genre,
                  'text': most_prolific.loc[most_prolific['favored_genre']==genre,'actor'],
                  'marker':{'size':10,'opacity':0.7,
                            'line':{'width':1.25,'color':'black'}}})
layout={'title':'Actors favored genres',
       'xaxis':{'title':'mean year of activity'},
       'yaxis':{'title':'mean score'}}
fig=Figure(data=trace,layout=layout)
pyo.iplot(fig)

We can also use this data to highlight single actors. Let us take a look at actors for who we have data of more than 20 movies.

In [ ]:
selection = df_appearance['title_year'] > 20
most_prolific = df_actors[selection]
most_prolific

## Part 4: Director Analysis
We start the actual analysis by computing the average per movie and total revenue of the directors. We only took into account the directs for which we have at least 4 movies as observations, to exclude extreme outliers. Not surprisingly, the top rated directors are probably directors you have heard about.

In [ ]:
df3 = df

def create_comparison_database(name, value, x, no_films):
    
    comparison_df = df3.groupby(name, as_index=False)
    
    if x == 'mean':
        comparison_df = comparison_df.mean()
    elif x == 'median':
        comparison_df = comparison_df.median()
    elif x == 'sum':
        comparison_df = comparison_df.sum() 
    
    # Create database with either name of directors or actors, the value being compared i.e. 'revenue',
    # and number of films they're listed with. Then sort by value being compared.
    name_count_key = df[name].value_counts().to_dict()
    comparison_df['films'] = comparison_df[name].map(name_count_key)
    comparison_df.sort_values(value, ascending=False, inplace=True)
    comparison_df[name] = comparison_df[name].map(str) + " (" + comparison_df['films'].astype(str) + ")"
   # create a Series with the name as the index so it can be plotted to a subgrid
    comp_series = comparison_df[comparison_df['films'] >= no_films][[name, value]][10::-1].set_index(name).ix[:,0]
    
    return comp_series

fig = plt.figure(figsize=(18,6))

# Director_name
plt.subplot2grid((2,3),(0,0), rowspan = 2)
create_comparison_database('director_name','revenue','sum', 4).plot(kind='barh', color='#006600')
plt.legend().set_visible(False)
plt.title("Total Revenue for Directors with 4+ Films")
plt.ylabel("Director (no. films)")
plt.xlabel("Revenue (in billons)")

plt.subplot2grid((2,3),(0,1), rowspan = 2)
create_comparison_database('director_name','revenue','mean', 4).plot(kind='barh', color='#ffff00')
plt.legend().set_visible(False)
plt.title('Average revenue for Directors with 4+ Films')
plt.ylabel("Director (no. films)")
plt.xlabel("Revenue (in billons)")

plt.tight_layout()

Notice how many of the directors that have a very high average budget per movie were nowhere to be seen in the revenue plot. Implying that, although they make expensive movies, they don't make the most grossing movies. Also note that a lot of high scoring directors are not found in the top ten highest budgeted directors. This implies that a big budget doesn't necessarily lead to a good, or well-received, movie. On the other hand, it shows that some directors, for instance Hayao Miyazaki, is capable of creating excellent movies with needing a very high budget. 

Now, all of this is of course only true for directors with 4+ movies. It is possible that directors with few movies were lucky. A question to ask the dataset could be whether there exist any directors that are capable of consistently creating well-received movies, without the need for big budgets. To answer this question we plot the average budget next to the average score per director, for directors with at least 15 movies.

In [ ]:
fig = plt.figure(figsize=(18,6))

# Director_name
plt.subplot2grid((2,3),(0,0), rowspan = 2)
create_comparison_database('director_name','budget','mean', 10).plot(kind='barh', color='#006600')
plt.legend().set_visible(False)
plt.title("Average budget for Directors with 15+ Filmss")
plt.ylabel("Director (no. films)")
plt.xlabel("Budget (in billons)")

plt.subplot2grid((2,3),(0,1), rowspan = 2)
create_comparison_database('director_name','vote_average','mean', 10).plot(kind='barh', color='#ffff00')
plt.legend().set_visible(False)
plt.title('Mean IMDB Score for Directors with 15+ Films')
plt.ylabel("Director (no. films)")

plt.xlabel("IMDB Score")
plt.xlim(0,10)

plt.tight_layout()

Now, we easily see that the two bar plots have more directors in common. Still, there are some directors who manage to create excellent movies without the need for a big budget. A funny observation is Michael Bay. While he is easily the king of budget, he is nowhere to be found in the top ten highest scoring directors.

Resources:

https://www.kaggle.com/fabiendaniel/film-recommendation-engine

https://www.kaggle.com/fabiendaniel/categorizing-actors-hands-on-plotly

https://www.kaggle.com/willacy/director-and-actor-s-total-gross-and-imdb-score

https://www.kaggle.com/fabiendaniel/choropleth-map-with-plot